In [1]:
!pip install openai==0.27.8
!pip install tiktoken
!pip install langchain
!pip install hnswlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 906.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata 

In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [3]:
# imports
import openai
import hnswlib
import langchain
from langchain.text_splitter import TextSplitter, CharacterTextSplitter
import PyPDF2
import requests

In [4]:
openai.api_key = ""
openai_params = {"model":"gpt-4-1106-preview",
                 "temperature":0.5,
                 "frequency_penalty":0.0,
                 "presence_penalty":0.0,
                 "max_tokens":1500,
                 "top_p":1}


In [5]:
def generate_answer(prompt,openai_params):
  message = [{"role":"user","content":prompt}]
  response = openai.ChatCompletion.create(messages=message,
                                        **openai_params)

  return response.choices[0].message.content

In [6]:
def tokenize_text_gpt(content,chunk_size=120,splitter_pattern=""):
    """
    Tokenize the text according to openai tokenizer using Langchain
    :param content:
    :return:
    """
    if not splitter_pattern:

        if "\n\n" in content:

            text_splitter_ = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=0,encoding_name="cl100k_base")
        elif "\n" in content:
            text_splitter_ = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=0,
                                                                         separator="\n",encoding_name="cl100k_base")
        else:
            text_splitter_ = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=0,
                                                                         separator=" ",encoding_name="cl100k_base")
    else:
        text_splitter_ = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size,chunk_overlap=0,
                                                                    separator=splitter_pattern,encoding_name="cl100k_base")
    passages = text_splitter_.split_text(content)

    return passages


# tokenized_text = tokenize_text_by_page(text)
# for page, tokens in tokenized_text.items():
#     print(f"Page {page}: {tokens}\n")

In [7]:
def extract_text_from_pdf(pdf_file_path):
    # Open the PDF file
    with open(pdf_file_path, 'rb') as file:
        # Create PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)

        extracted_text = ""

        for page in pdf_reader.pages:

            extracted_text += page.extract_text()

        return extracted_text

In [8]:
def create_index2(text_chunks):

    embeddings = get_embedding_list(text_chunks)
    if len(embeddings) == 0:
        print("No embeddings generated.")
        return None, {}

    dimension = len(embeddings[0])  # Dynamically get the dimension of embeddings

    index1 = hnswlib.Index(space='l2', dim=dimension)
    index1.init_index(max_elements=len(text_chunks), ef_construction=200, M=16)

    # Bulk adding to the index
    index1.add_items(embeddings)

    index1.set_ef(50)
    return index1

In [9]:
def get_embedding_list(texts, model="text-embedding-ada-002"):
  texts = [re.sub("\n+", " ", text) for text in texts]
  embedding_data = openai.Embedding.create(input = texts, model=model)['data']
  print("embeddings returned from openai")
  return [embedding_data[i]["embedding"] for i in range(len(embedding_data))]



def get_embedding(text, model="text-embedding-ada-002"):
  text = re.sub("\n+", " ", text)
  return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [10]:
def search_similar_text2(query, index, top_k):
    query_vector = get_embedding(query)
    try:
        labels, distances = index.knn_query(query_vector, k=top_k)

        # Flatten the labels and distances since we have a single query
        labels = labels.flatten()
        distances = distances.flatten()

    except Exception as e:
        print(e)
        results = []
    return labels

In [11]:
text_dict = extract_text_from_pdf("/content/king09a.pdf")
chunks = tokenize_text_gpt(text_dict)

In [12]:
len(chunks)

25

In [13]:
import re
index = create_index2(chunks)

embeddings returned from openai


In [19]:
query = "What is Linear algebra?"

In [20]:
searched_index = search_similar_text2(query,index,10)

In [21]:
context = ""
for i in searched_index:
  context = context + " " +chunks[i] + "\n\n"

In [22]:
def qa_prompt(instruction, context):
    _prompt = f"""When responding to instructions, always ensure your answers:
1.Summarize key information from the provided documents to address the task directly.
2.Identify and resolve any inconsistencies or inaccuracies between the instruction and the document content.
3.Emphasize important details, removing unnecessary information for clarity.
4.Note any missing content, suggesting alternatives or additional sources as necessary.
5.Maintain accuracy in accordance with the document(s) content and ethical standards.
6.Provide a concise summary of relevant insights from the document(s) related to the instruction.
7.Include references with PDF names and page numbers for relevant excerpts that support your response.
8.Tailor your response to the nature of the instruction, focusing on accuracy and relevance.

Context:{context}
Instruction:{instruction}
"""
    return _prompt


In [23]:
fin_prompt = qa_prompt(query, context)
generate_answer(fin_prompt,openai_params)

'Linear algebra is a branch of mathematics that deals with vectors, vector spaces, linear mappings, and systems of linear equations. It is fundamental to many areas of mathematics and its applications in science and engineering. In the context of the document "DLIB-ML: A MACHINE LEARNING TOOLKIT," the linear algebra component of the dlib-ml library provides a set of core functionalities that are essential for implementing various machine learning tools.\n\nThe linear algebra component in dlib-ml is designed using template expression techniques, which allow for writing simple, Matlab-like expressions that compile into efficient C code. This design approach makes the dlib-ml library generic enough to operate on any kind of structured data, such as column vectors, images, or other forms, as long as an appropriate kernel is provided. Furthermore, dlib-ml can utilize optimized libraries like ATLAS or Intel MKL for improved performance while maintaining simple syntax for operations like matr